[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/pixeltable/pixeltable/blob/master/docs/release/tutorials/working-with-openai.ipynb)&nbsp;&nbsp;
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixeltable/pixeltable/blob/master/docs/release/tutorials/working-with-openai.ipynb)

# Working with OpenAI in Pixeltable

Pixeltable unifies data and computation into a table interface. In the [Pixeltable Basics](https://pixeltable.github.io/pixeltable/tutorials/pixeltable-basics/) tutorial, we saw how OpenAI API calls can be incorporated into Pixeltable workflows. In this tutorial, we'll go into more depth on OpenAI integration. You'll need an OpenAI API key to run this demo.

### Prerequisites
- An OpenAI account with an API key (https://openai.com/index/openai-api/)

### Important Notes

- OpenAI usage may incur costs based on your OpenAI plan.
- Be mindful of sensitive data and consider security measures when integrating with external services.

In [1]:
%pip install -q pixeltable openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pixeltable as pxt
pxt.create_dir('demo', ignore_errors=True)

Connected to Pixeltable database at: postgresql://postgres:@/pixeltable?host=/Users/orm/.pixeltable/pgdata


## Creating the Table

First, we'll create a table and populate it with some sample data.

In [3]:
pxt.drop_table('demo.openai', ignore_errors=True)
t = pxt.create_table('demo.openai', {'id': pxt.IntType(), 'input': pxt.StringType()})

Created table `openai`.


In [4]:
# text from https://en.wikipedia.org/wiki/Global_financial_crisis_in_September_2008

wikipedia_text = '''On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.
The significance of the Lehman Brothers bankruptcy is disputed with some assigning it a pivotal role in the unfolding of subsequent events.
The principals involved, Ben Bernanke and Henry Paulson, dispute this view, citing a volume of toxic assets at Lehman which made a rescue impossible.[16][17] Immediately following the bankruptcy, JPMorgan Chase provided the broker dealer unit of Lehman Brothers with $138 billion to "settle securities transactions with customers of Lehman and its clearance parties" according to a statement made in a New York City Bankruptcy court filing.[18]
The same day, the sale of Merrill Lynch to Bank of America was announced.[19] The beginning of the week was marked by extreme instability in global stock markets, with dramatic drops in market values on Monday, September 15, and Wednesday, September 17.
On September 16, the large insurer American International Group (AIG), a significant participant in the credit default swaps markets, suffered a liquidity crisis following the downgrade of its credit rating.
The Federal Reserve, at AIG's request, and after AIG had shown that it could not find lenders willing to save it from insolvency, created a credit facility for up to US$85 billion in exchange for a 79.9% equity interest, and the right to suspend dividends to previously issued common and preferred stock.[20]'''

sample_inputs = wikipedia_text.split('\n')

In [5]:
# Insert a single sample row into the table
t.insert(id=0, input=sample_inputs[0])
t.show()

Inserting rows into `openai`: 0 rows [00:00, ? rows/s]

Inserting rows into `openai`: 1 rows [00:00, 931.24 rows/s]


Inserted 1 row with 0 errors.


id,input
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers."


## Making OpenAI API calls

Calling OpenAI API endpoints involves constructing a message object, which we can express in Pixeltable by adding a new computed column.

In [6]:
prompt = "For the following sentence, extract all company names from the text."

msgs = [
    { "role": "system", "content": prompt },
    { "role": "user", "content": t.input }
]

t.add_column(input_msgs=msgs)

Computing cells:   0%|                                                    | 0/1 [00:00<?, ? cells/s]

Computing cells: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 572.13 cells/s]


Added 1 column value with 0 errors.


UpdateStatus(num_rows=1, num_computed_values=1, num_excs=0, updated_cols=[], cols_with_excs=[])

Unlike the values of the`input` column, which users provide, the `t.input_msgs` column is computed automatically from the `t.input` column values:

In [7]:
t.show()

id,input,input_msgs
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","[{'role': 'system', 'content': 'For the following sentence, extract all company names from the text.'}, {'role': 'user', 'content': 'On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.'}]"


In Pixeltable, OpenAI API calls are exposed as Pixeltable functions, which can be used to create computed columns. You'll need to enter your OpenAI API key now.

In [8]:
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

Now we can run the prompt against our input data using OpenAI's `chat_completions` API.

In [9]:
from pixeltable.functions import openai

t['chat_output'] = openai.chat_completions(model='gpt-3.5-turbo', messages=t.input_msgs)
t.show()

Computing cells:   0%|                                                    | 0/1 [00:00<?, ? cells/s]

Computing cells: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.49 cells/s]


Added 1 column value with 0 errors.


id,input,input_msgs,chat_output
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","[{'role': 'system', 'content': 'For the following sentence, extract all company names from the text.'}, {'role': 'user', 'content': 'On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.'}]","{'id': 'chatcmpl-9Su4S630t8UQ2pRJpY1Ho9lfxyMXh', 'model': 'gpt-3.5-turbo-0125', 'usage': {'total_tokens': 66, 'prompt_tokens': 61, 'completion_tokens': 5}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '1. Lehman Brothers', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1716675748, 'system_fingerprint': None}"


The output of the OpenAI API calls are generally complex JSON structures, which require some navigation to extract the response. We can express this as JSON path expressions and create another computed column:

In [10]:
t['response'] = t.chat_output.choices[0].message.content
t.show()

Computing cells:   0%|                                                    | 0/1 [00:00<?, ? cells/s]

Computing cells: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 462.28 cells/s]


Added 1 column value with 0 errors.


id,input,input_msgs,chat_output,response
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","[{'role': 'system', 'content': 'For the following sentence, extract all company names from the text.'}, {'role': 'user', 'content': 'On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.'}]","{'id': 'chatcmpl-9Su4S630t8UQ2pRJpY1Ho9lfxyMXh', 'model': 'gpt-3.5-turbo-0125', 'usage': {'total_tokens': 66, 'prompt_tokens': 61, 'completion_tokens': 5}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '1. Lehman Brothers', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1716675748, 'system_fingerprint': None}",1. Lehman Brothers


Let's run a query to look only at the input and output:

In [11]:
t.select(t.input, t.response).show()

input,response
"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.",1. Lehman Brothers


Once we have defined these computed columns, much like with a spreadsheet, newly inserted `t.input` values trigger computation of all derived columns, such as `t.response`. Let's insert the rest of our data and see how this works.

In [12]:
t.insert({'id': i, 'input': sample_inputs[i]} for i in range(1, len(sample_inputs)))

Computing cells:   0%|                                                   | 0/15 [00:00<?, ? cells/s]

Inserting rows into `openai`: 0 rows [00:00, ? rows/s]

Inserting rows into `openai`: 5 rows [00:00, 1303.39 rows/s]


Computing cells: 100%|██████████████████████████████████████████| 15/15 [00:02<00:00,  5.34 cells/s]


Inserted 5 rows with 0 errors.


UpdateStatus(num_rows=5, num_computed_values=15, num_excs=0, updated_cols=[], cols_with_excs=[])

In [13]:
t.select(t.input, t.response).show()

input,response
"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.",1. Lehman Brothers
The significance of the Lehman Brothers bankruptcy is disputed with some assigning it a pivotal role in the unfolding of subsequent events.,Lehman Brothers
"The principals involved, Ben Bernanke and Henry Paulson, dispute this view, citing a volume of toxic assets at Lehman which made a rescue impossible.[16][17] Immediately following the bankruptcy, JPMorgan Chase provided the broker dealer unit of Lehman Brothers with $138 billion to ""settle securities transactions with customers of Lehman and its clearance parties"" according to a statement made in a New York City Bankruptcy court filing.[18]",1. Lehman Brothers\n2. JPMorgan Chase
"The same day, the sale of Merrill Lynch to Bank of America was announced.[19] The beginning of the week was marked by extreme instability in global stock markets, with dramatic drops in market values on Monday, September 15, and Wednesday, September 17.",- Merrill Lynch\n- Bank of America
"On September 16, the large insurer American International Group (AIG), a significant participant in the credit default swaps markets, suffered a liquidity crisis following the downgrade of its credit rating.",American International Group (AIG)
"The Federal Reserve, at AIG's request, and after AIG had shown that it could not find lenders willing to save it from insolvency, created a credit facility for up to US$85 billion in exchange for a 79.9% equity interest, and the right to suspend dividends to previously issued common and preferred stock.[20]",1. The Federal Reserve\n2. AIG


## Adding Ground Truth Data

Now let's see how Pixeltable can be used to evaluate a model against ground truth data. We'll start by manually populating a `ground_truth` column in our table.

In [14]:
t['ground_truth'] = pxt.StringType(nullable=True)

ground_truth = [
    'Lehman Brothers',
    'Lehman Brothers',
    'JP Morgan Chase, Lehman Brothers',
    'Merill Lynch, Bank of America',
    'American International Group',
    'American International Group',
]

for i, gt in enumerate(ground_truth):
    t.update({'ground_truth': gt}, where=(t.id == i))

Added 6 column values with 0 errors.


Inserting rows into `openai`: 0 rows [00:00, ? rows/s]

Inserting rows into `openai`: 1 rows [00:00, 1216.80 rows/s]

Inserting rows into `openai`: 0 rows [00:00, ? rows/s]

Inserting rows into `openai`: 1 rows [00:00, 2136.68 rows/s]

...

Inserting rows into `openai`: 1 rows [00:00, 2432.89 rows/s]

Inserting rows into `openai`: 0 rows [00:00, ? rows/s]

Inserting rows into `openai`: 1 rows [00:00, 2777.68 rows/s]

Inserting rows into `openai`: 0 rows [00:00, ? rows/s]

Inserting rows into `openai`: 1 rows [00:00, 2855.21 rows/s]

And this is what we have so far:

In [15]:
t.select(t.input, t.response, t.ground_truth).show()

input,response,ground_truth
"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.",1. Lehman Brothers,Lehman Brothers
The significance of the Lehman Brothers bankruptcy is disputed with some assigning it a pivotal role in the unfolding of subsequent events.,Lehman Brothers,Lehman Brothers
"The principals involved, Ben Bernanke and Henry Paulson, dispute this view, citing a volume of toxic assets at Lehman which made a rescue impossible.[16][17] Immediately following the bankruptcy, JPMorgan Chase provided the broker dealer unit of Lehman Brothers with $138 billion to ""settle securities transactions with customers of Lehman and its clearance parties"" according to a statement made in a New York City Bankruptcy court filing.[18]",1. Lehman Brothers\n2. JPMorgan Chase,"JP Morgan Chase, Lehman Brothers"
"The same day, the sale of Merrill Lynch to Bank of America was announced.[19] The beginning of the week was marked by extreme instability in global stock markets, with dramatic drops in market values on Monday, September 15, and Wednesday, September 17.",- Merrill Lynch\n- Bank of America,"Merill Lynch, Bank of America"
"The Federal Reserve, at AIG's request, and after AIG had shown that it could not find lenders willing to save it from insolvency, created a credit facility for up to US$85 billion in exchange for a 79.9% equity interest, and the right to suspend dividends to previously issued common and preferred stock.[20]",1. The Federal Reserve\n2. AIG,American International Group
"On September 16, the large insurer American International Group (AIG), a significant participant in the credit default swaps markets, suffered a liquidity crisis following the downgrade of its credit rating.",American International Group (AIG),American International Group


## Evaluation

Now that we have some ground truth available, we can carry out basic evaluations of the GPT outputs, in this case by asking ChatGPT to decide whether the two are equivalent. 

To start with, we'll create an evaluation prompt. In this case, the prompt requires some bespoke string substitution, so it's easiest to do using a UDF. (See the [Pixeltable Basics](https://pixeltable.github.io/pixeltable/tutorials/pixeltable-basics/) tutorial and the [UDFs in Pixeltable](https://pixeltable.github.io/pixeltable/howto/udfs-in-pixeltable/) guide for more details on UDFs.)

In [16]:
system_prompt = '''
Compare the following listA and listB of entities, and check if they contain the same entities.
Return a JSON object with the following format:
{"reasoning": explaining your reasoning, "decision": 1 if the lists matched, 0 otherwise}
'''

@pxt.udf
def eval_prompt(listA: str, listB: str) -> list[dict]:
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': f'listA: "{listA}" \n listB: "{listB}"'}
    ]

t['eval_prompt'] = eval_prompt(t.response, t.ground_truth)

Computing cells:   0%|                                                    | 0/6 [00:00<?, ? cells/s]

Computing cells: 100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 1552.01 cells/s]


Added 6 column values with 0 errors.


In [17]:
t.select(t.eval_prompt).show()

eval_prompt
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a JSON object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""1. Lehman Brothers"" listB: ""Lehman Brothers""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a JSON object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""- Merrill Lynch - Bank of America"" listB: ""Merill Lynch, Bank of America""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a JSON object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""1. The Federal Reserve 2. AIG"" listB: ""American International Group""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a JSON object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""Lehman Brothers"" listB: ""Lehman Brothers""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a JSON object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""1. Lehman Brothers 2. JPMorgan Chase"" listB: ""JP Morgan Chase, Lehman Brothers""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a JSON object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""American International Group (AIG)"" listB: ""American International Group""'}]"


The actual evaluation happens in another computed column. We can use OpenAI's handy `response_format` parameter to enforce that the output is properly formed JSON.

In [18]:
t['eval'] = openai.chat_completions(model='gpt-3.5-turbo', messages=t.eval_prompt, response_format={'type': 'json_object'})
t['eval_output'] = t.eval.choices[0].message.content

Computing cells:   0%|                                                    | 0/6 [00:00<?, ? cells/s]

Computing cells: 100%|████████████████████████████████████████████| 6/6 [00:06<00:00,  1.03s/ cells]


Added 6 column values with 0 errors.


Computing cells:   0%|                                                    | 0/6 [00:00<?, ? cells/s]

Computing cells: 100%|███████████████████████████████████████████| 6/6 [00:00<00:00, 947.40 cells/s]


Added 6 column values with 0 errors.


Let's take a look:

In [19]:
t.select(t.response, t.ground_truth, t.eval_output).show()

response,ground_truth,eval_output
1. Lehman Brothers,Lehman Brothers,"{""reasoning"": ""The entities in listA and listB are the same, as '1.' is just a numbering and can be ignored when comparing the entities."", ""decision"": 1}"
1. The Federal Reserve\n2. AIG,American International Group,"{\n ""reasoning"": ""The entities in listA are 'The Federal Reserve' and 'AIG', while the entity in listB is 'American International Group'. Since 'AIG' (American International Group) appears in both lists, we can consider them as matching even though the formatting differs."",\n ""decision"": 1\n}"
1. Lehman Brothers\n2. JPMorgan Chase,"JP Morgan Chase, Lehman Brothers","{""reasoning"": ""The lists do not match because the entities are in a different order in listA and listB. Additionally, there is a slight difference in the naming convention (e.g., 'JPMorgan Chase' in listA vs. 'JP Morgan Chase' in listB)."", ""decision"": 0}"
- Merrill Lynch\n- Bank of America,"Merill Lynch, Bank of America","{\n ""reasoning"": ""The entities in listA are separated with a hyphen while the entities in listB are separated with a comma. However, the entities themselves are the same."",\n ""decision"": 1\n}"
Lehman Brothers,Lehman Brothers,"{""reasoning"": ""The entities in both lists are exactly the same."", ""decision"": 1}"
American International Group (AIG),American International Group,"{""reasoning"": ""The entities in listA and listB refer to the same company, with 'AIG' being an abbreviation of 'American International Group'. Therefore, the lists contain the same entity."", ""decision"": 1}"


Finally, it's time to pull the `decision` out of the JSON structs returned by OpenAI. There's just one complication: the `chat_completions` responses are strings, not JSON structs. We can resolve this with Pixeltable's handy `apply` method, which turns any Python function into a Pixeltable function. In this case, we'll apply the Python function `json.loads` to parse the string into a JSON struct.

In [20]:
import json

t['eval_decision'] = t.eval_output.apply(json.loads).decision

Computing cells:   0%|                                                    | 0/6 [00:00<?, ? cells/s]

Computing cells: 100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 1314.07 cells/s]


Added 6 column values with 0 errors.


In [21]:
t.select(t.response, t.ground_truth, t.eval_output, t.eval_decision).show()

response,ground_truth,eval_output,eval_decision
1. Lehman Brothers,Lehman Brothers,"{""reasoning"": ""The entities in listA and listB are the same, as '1.' is just a numbering and can be ignored when comparing the entities."", ""decision"": 1}",1
1. The Federal Reserve\n2. AIG,American International Group,"{\n ""reasoning"": ""The entities in listA are 'The Federal Reserve' and 'AIG', while the entity in listB is 'American International Group'. Since 'AIG' (American International Group) appears in both lists, we can consider them as matching even though the formatting differs."",\n ""decision"": 1\n}",1
1. Lehman Brothers\n2. JPMorgan Chase,"JP Morgan Chase, Lehman Brothers","{""reasoning"": ""The lists do not match because the entities are in a different order in listA and listB. Additionally, there is a slight difference in the naming convention (e.g., 'JPMorgan Chase' in listA vs. 'JP Morgan Chase' in listB)."", ""decision"": 0}",0
- Merrill Lynch\n- Bank of America,"Merill Lynch, Bank of America","{\n ""reasoning"": ""The entities in listA are separated with a hyphen while the entities in listB are separated with a comma. However, the entities themselves are the same."",\n ""decision"": 1\n}",1
Lehman Brothers,Lehman Brothers,"{""reasoning"": ""The entities in both lists are exactly the same."", ""decision"": 1}",1
American International Group (AIG),American International Group,"{""reasoning"": ""The entities in listA and listB refer to the same company, with 'AIG' being an abbreviation of 'American International Group'. Therefore, the lists contain the same entity."", ""decision"": 1}",1
